In [ ]:
import json
import pandas as pd
import requests
import sqlalchemy
import datetime
def lambda_handler(event, context):

    
    schema="gans"
    host="wbs-project3-db1.c5v1hsxt2ic8.eu-central-1.rds.amazonaws.com"
    user="admin"
    password="Awspark2022"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
 
    cities_df = pd.read_sql('city',con=con)
    
    # weather_df = get_weather(cities_df)
    # weather_df.to_sql('weather', con=con, if_exists='append',index=False)
 
    # df = pd.DataFrame({'FirstName': ['Lambda'], 'City': ['Cloud']})
 
    # df.to_sql('test_table',if_exists='append',con=con,index=False)
 
    airports_df =pd.read.sql('airports', con=con)
 
    flights_df = get_flights(airports_df)
    flights_df.to_sql('flights', con=con, if_exists='append', index= False)
 
 
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }
    
    def get_weather(city_df):
    
        API_key = '29f3c126993ad4136d3ad1375c76a0be'
    
        weather_dict = {'city_id':[],
                'time':[],
                'temp':[],
                'wind_speed':[]}
    
        for i, city in enumerate(city_df['city_id']):
            url = f"http://api.openweathermap.org/data/2.5/forecast?lat={city_df.iloc[i]['lat']}&lon={city_df.iloc[i]['long']}&appid={API_key}&units=metric"
            response = requests.get(url)
            weather_json = response.json()
            for time in weather_json['list'][:9]:
                weather_dict['city_id'].append(city)
                weather_dict['time'].append(time['dt_txt'])
                weather_dict['temp'].append(time['main']['temp'])
                weather_dict['wind_speed'].append(time['wind']['speed'])

    weather_df = pd.DataFrame(weather_dict)
   
    return weather_df
    
    weather_df = get_weather(cities_df)
    weather_df.to_sql('weather', con=con, if_exists='append',index=False)
    
    
    
    def get_flights(airports_df):
        querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true",
                       "withPrivate":"true","withLocation":"false"}
        headers = {
            "X-RapidAPI-Key": "0139f8dfbdmsh2c74d8d3164ce7ap1b3e2bjsn77505670c3ca",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        today = datetime.date.today()
        tomorrow = str(today + datetime.timedelta(days=1))
        
        times = [["00:00","11:59"],["12:00","23:59"]]
        
        icao_list = airports_df['icao'].to_list()
        
        flights_dict = {'ICAO':[],'departure_airport':[],'local_time':[]}
        
    
        for time in times:
    
            for icao in icao_list[1:2]:
                url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
                response = requests.request("GET", url, headers=headers, params=querystring)
                print(response)
                flight_json = response.json()
    
                for flight in flight_json['arrivals']:
                    flights_dict['ICAO'].append(icao)
                    try:
                        flights_dict['departure_airport'].append(flight['departure']['airport']['icao'])
                    except:
                        flights_dict['departure_airport'].append('unknown')
                    try:
                        flights_dict['local_time'].append(flight['arrival']['scheduledTimeLocal'])
                    except:
                        flights_dict['local_time'].append(pd.NaT)
                        
        flights_df = pd.DataFrame(flights_dict)
        flights_df['local_time'] = pd.to_datetime(flights_df['local_time'])
        
        return flights_df